In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf_stock
from utils.fetch_utils import fetch_exchange_rate
import requests
from datetime import date, timedelta, datetime
import json
import yfinance as yf
from utils.other_utils import round_half_up, xirr
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', None)

In [ ]:
dt = "10-01-2025"
num_date = datetime.strptime(dt, "%d-%m-%Y")
ref_date= datetime.strptime(dt, "%d-%m-%Y")
print(num_date)



2025-01-10 00:00:00


In [12]:
import numpy as np
from datetime import date
from scipy.optimize import newton



# --- Esempio di Utilizzo ---

# I flussi di cassa, con l'investimento iniziale come valore NEGATIVO.
# (Esempio: Investi 10.000, ricevi 2.000, 3.000 e 7.000)
flussi=[-425.68, -200.0, -100.0, -300.0, -50.0, 104.2, -150.0, -20.0, 90.0, -95.0, -55.0, 558.39, 893.37]

# Le date precise in cui i flussi si sono verificati
date_flussi = [
    date(2025, 3, 3),
    date(2025, 5, 30), # Data Investimento (T0)
    date(2025, 5, 30), # Data Investimento (T0)
    date(2025, 6, 17), # Data Flusso 1 (dopo 6 mesi)
    date(2025, 8, 4), # Data Flusso 2 (dopo 1 anno)
    date(2025, 8, 18), # Data Flusso 3 (dopo 1 anno e 9 mesi)
    date(2025, 9, 23), # Data Flusso 2 (dopo 1 anno)
    date(2025, 10, 1), # Data Flusso 2 (dopo 1 anno)
    date(2025, 10, 9),
    date(2025, 10, 15), # Data Flusso 2 (dopo 1 anno)
    date(2025, 10, 16), # Data Flusso 2 (dopo 1 anno)
    date(2025, 10, 31), # Data Flusso 2 (dopo 1 anno)
    date(2025, 10, 31) # Data Flusso 2 (dopo 1 anno) 
]

period=((date(2025, 10, 31) - date(2025, 3, 3)).days)
print(period)
# Calcola il TIR Esteso
xirr_result = xirr(flussi, date_flussi, annualization=period)

# Stampa il risultato
print("\nRisultato XIRR:")
print(f"Flussi di Cassa: {flussi}")
print(f"Date: {date_flussi}")
print(f"XIRR (Tasso Interno di Rendimento Esteso): {xirr_result * 100:.2f}%")




242

Risultato XIRR:
Flussi di Cassa: [-425.68, -200.0, -100.0, -300.0, -50.0, 104.2, -150.0, -20.0, 90.0, -95.0, -55.0, 558.39, 893.37]
Date: [datetime.date(2025, 3, 3), datetime.date(2025, 5, 30), datetime.date(2025, 5, 30), datetime.date(2025, 6, 17), datetime.date(2025, 8, 4), datetime.date(2025, 8, 18), datetime.date(2025, 9, 23), datetime.date(2025, 10, 1), datetime.date(2025, 10, 9), datetime.date(2025, 10, 15), datetime.date(2025, 10, 16), datetime.date(2025, 10, 31), datetime.date(2025, 10, 31)]
XIRR (Tasso Interno di Rendimento Esteso): 32.21%


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

# --- 1. Codice fornito dall'utente per ottenere l'indice delle date target ---

total_tickers = ['POET', 'AMD', 'QUBT', 'ARM', 'MWEQ.MI', 'AASI.MI', 'AVGO', 'QBTS', 'TSM', 'IONQ']

# Scarica i dati sui prezzi
try:
    prices_df_raw = yf.download(total_tickers, start="2010-01-01", end="2025-10-24")
    
    # Assicurati che 'prices_df' non sia vuoto o completamente NaN
    if not prices_df_raw.empty:
        # Estrai solo i prezzi 'Close'
        prices_df = prices_df_raw["Close"]
        
        # Gestisce il caso di un singolo ticker (restituisce una Serie)
        if isinstance(prices_df, pd.Series):
            prices_df = prices_df.to_frame(name=total_tickers[0] if len(total_tickers) == 1 else "Close")
            
        # Rimuovi righe con NaN (questo definisce le date valide)
        prices_df = prices_df.dropna()
        
        # Questo è l'indice che useremo per allineare tutto
        target_index = prices_df.index
        
        print(f"Indice date target creato con successo da prices_df. Numero di date: {len(target_index)}")
        if target_index.empty:
            print("Attenzione: l'indice target di prices_df è vuoto dopo dropna().")
            target_index = pd.DatetimeIndex([])
    else:
        print("Download di yfinance non ha restituito dati.")
        target_index = pd.DatetimeIndex([])

except Exception as e:
    print(f"Errore durante il download da yfinance: {e}")
    target_index = pd.DatetimeIndex([])


# --- 2. Caricamento ed elaborazione di portfolio_df.csv ---

# Nome del file fornito
portfolio_file = "/home/atonello/Downloads/portfolio_df.csv"

try:
    portfolio_data = pd.read_csv(portfolio_file, parse_dates=['Data'])
    portfolio_data = portfolio_data.sort_values(by='Data')

    # --- 3. Preparazione dei dati sparsi ---

    # Isola i dati sulla liquidità
    liquidity_sparse = portfolio_data.dropna(subset=['Liquidita Totale'])
    liquidity_sparse = liquidity_sparse.drop_duplicates(subset=['Data'], keep='last')
    liquidity_sparse = liquidity_sparse.set_index('Data')[['Liquidita Totale']]
    liquidity_sparse = liquidity_sparse.rename(columns={'Liquidita Totale': 'Liquidita'})

    # Isola i dati delle quantità dei ticker
    quantities_sparse = portfolio_data.dropna(subset=['Ticker', 'QT. Totale'])
    quantities_sparse = quantities_sparse.drop_duplicates(subset=['Data', 'Ticker'], keep='last')
    
    # Esegui il pivot per avere i ticker come colonne
    quantities_wide_sparse = quantities_sparse.pivot(index='Data', columns='Ticker', values='QT. Totale')

    # --- 4. Unione dei dati sparsi e riempimento ---

    combined_sparse_data = pd.concat([quantities_wide_sparse, liquidity_sparse], axis=1)

    for ticker in total_tickers:
        if ticker not in combined_sparse_data.columns:
            combined_sparse_data[ticker] = np.nan
            
    if 'Liquidita' not in combined_sparse_data.columns:
        combined_sparse_data['Liquidita'] = np.nan

    final_columns = total_tickers + ['Liquidita']
    combined_sparse_data = combined_sparse_data[final_columns]

    # --- 5. Allineamento all'indice di prices_df ---

    if not target_index.empty:
        # Combina gli indici per un ffill corretto
        combined_index = target_index.union(combined_sparse_data.index).sort_values()

        # Reindicizza ed esegui il forward fill
        quantities_df_filled = combined_sparse_data.reindex(combined_index).ffill()

        # Seleziona solo le date di prices_df
        final_quantities_df = quantities_df_filled.loc[target_index]

        # Riempi i NaN dei ticker con 0
        final_quantities_df[total_tickers] = final_quantities_df[total_tickers].fillna(0)
        
        # (Opzionale: riempi Liquidita NaN con 0 se preferito, altrimenti ffill dovrebbe averla gestita)
        # final_quantities_df['Liquidita'] = final_quantities_df['Liquidita'].fillna(0)


        # --- 6. Aggiunta Valore Titoli e NAV ---
        
        # 'prices_df' è già il DataFrame dei prezzi Close allineato a target_index
        # 'final_quantities_df' è il DataFrame delle quantità allineato a target_index
        
        # Assicurati che i prezzi siano nel formato corretto (DataFrame)
        if isinstance(prices_df, pd.Series):
            prices_df_for_calc = prices_df.to_frame(name=total_tickers[0])
        else:
            # Assicurati che le colonne siano solo i ticker
            prices_df_for_calc = prices_df[total_tickers]
            
        # Estrai le colonne delle quantità
        quantities_for_calc = final_quantities_df[total_tickers]
        
        # 1. Calcola Valore Titoli
        # Moltiplicazione Element-wise e somma per riga
        final_quantities_df['Valore Titoli'] = (prices_df_for_calc * quantities_for_calc).sum(axis=1)

        # 2. Calcola NAV
        final_quantities_df['NAV'] = final_quantities_df['Valore Titoli'] + final_quantities_df['Liquidita']

        # --- 7. Output ---
        print("\n--- DataFrame Finale ('final_quantities_df') con NAV ---")
        print(final_quantities_df.head())
        print("\nInformazioni sul DataFrame:")
        final_quantities_df.info()
        final_quantities_df.to_csv("/home/atonello/Downloads/portfolio_df_final.csv")

    else:
        print("\nImpossibile creare il DataFrame finale perché l'indice di 'prices_df' è vuoto.")
        # Crea un dataframe vuoto con le colonne corrette
        final_columns_complete = total_tickers + ['Liquidita', 'Valore Titoli', 'NAV']
        final_quantities_df = pd.DataFrame(columns=final_columns_complete)
        print(final_quantities_df)


except FileNotFoundError:
    print(f"Errore: File '{portfolio_file}' non trovato.")
except Exception as e:
    print(f"Si è verificato un errore imprevisto: {e}")

[*********************100%***********************]  10 of 10 completed

Indice date target creato con successo da prices_df. Numero di date: 258

--- DataFrame Finale ('final_quantities_df') con NAV ---
            POET  AMD  QUBT  ARM  MWEQ.MI  AASI.MI  AVGO  QBTS  TSM  IONQ  \
Date                                                                        
2024-10-04   0.0  0.0   0.0  0.0      0.0      0.0   0.0   0.0  0.0   0.0   
2024-10-07   0.0  0.0   0.0  0.0      0.0      0.0   0.0   0.0  0.0   0.0   
2024-10-08   0.0  0.0   0.0  0.0      0.0      0.0   0.0   0.0  0.0   0.0   
2024-10-09   0.0  0.0   0.0  0.0      0.0      0.0   0.0   0.0  0.0   0.0   
2024-10-10   0.0  0.0   0.0  0.0      0.0      0.0   0.0   0.0  0.0   0.0   

            Liquidita  Valore Titoli  NAV  
Date                                       
2024-10-04        NaN            0.0  NaN  
2024-10-07        NaN            0.0  NaN  
2024-10-08        NaN            0.0  NaN  
2024-10-09        NaN            0.0  NaN  
2024-10-10        NaN            0.0  NaN  

Informazioni sul Dat

In [ ]:
import pandas as pd

input_file = '/home/atonello/Downloads/portfolio_df.csv'
try:
    df = pd.read_csv(input_file)
except FileNotFoundError:
    print(f"Errore: File '{input_file}' non trovato.")
    exit()

df_total = df.copy()
accounts = df_total['Conto'].unique()
liq_cols = []

for acc in accounts:
    col_name = f'liq_running_{acc}'
    df_total[col_name] = df_total.where(df_total['Conto'] == acc)['Liquidita Attuale']
    df_total[col_name] = df_total[col_name].ffill()
    liq_cols.append(col_name)

df_total[liq_cols] = df_total[liq_cols].fillna(0)
df_total['Liquidita totale'] = df_total[liq_cols].sum(axis=1)

df_final = df_total.drop(columns=liq_cols)

output_file = '/home/atonello/Downloads/portfolio_df_cum.csv'
df_final.to_csv(output_file, index=False)

DataFrame con 'Liquidita totale' calcolata e salvato come '/home/atonello/Downloads/portfolio_df_cum.csv'.

Prime 5 righe del DataFrame generato:
         Data    Conto Ticker  QT. Attuale  Liquidita Attuale  \
0  2025-03-03   Fineco    NaN          NaN             425.68   
1  2025-03-03   Fineco    AMD          2.0             235.21   
2  2025-03-04   Fineco    ARM          2.0               0.00   
3  2025-05-30  BG Saxo    NaN          NaN             200.00   
4  2025-05-30  BG Saxo   IONQ          4.0              53.99   

   Liquidita totale  
0            425.68  
1            235.21  
2              0.00  
3            200.00  
4             53.99  

Verifica righe specifiche (basate su 0-indexing):

Riga 27:
          Data   Conto Ticker  QT. Attuale  Liquidita Attuale  \
27  2025-10-07  Fineco    ARM          0.0             576.83   

    Liquidita totale  
27            578.55  

Riga 28:
          Data   Conto Ticker  QT. Attuale  Liquidita Attuale  \
28  2025-10-07  Fi

In [ ]:
import pandas as pd
import numpy as np

input_file = '/home/atonello/Downloads/portfolio_df.csv'
try:
    df = pd.read_csv(input_file)
except FileNotFoundError:
    print(f"Errore: File '{input_file}' non trovato.")
    exit()

df_final = df.copy()
pairs = df.dropna(subset=['Ticker'])[['Conto', 'Ticker']].drop_duplicates().values

state_cols = []
ticker_to_cols_map = {}

# 3. Crea una colonna di stato "running" per ogni coppia (Conto, Ticker)
for conto, ticker in pairs:
    # Nome della colonna di stato (es. 'state_qty_Fineco_AMD')
    col_name = f'state_qty_{conto}_{ticker}'
    state_cols.append(col_name)
    
    # Mappa il ticker alla sua colonna di stato
    if ticker not in ticker_to_cols_map:
        ticker_to_cols_map[ticker] = []
    ticker_to_cols_map[ticker].append(col_name)
    
    # Crea la colonna di stato:
    # Prende il valore 'QT. Attuale' solo per la riga esatta
    mask = (df_final['Conto'] == conto) & (df_final['Ticker'] == ticker)
    df_final[col_name] = df_final.where(mask)['QT. Attuale']
    
    # Propaga l'ultimo valore valido (ffill)
    df_final[col_name] = df_final[col_name].ffill()

# 4. Riempi tutti i NaN iniziali (prima della prima transazione) con 0
df_final[state_cols] = df_final[state_cols].fillna(0)

# 5. Inizializza la colonna 'QT. totale' con NaN
df_final['QT. totale'] = np.nan

# 6. Calcola il totale sommando le colonne di stato appropriate per ogni riga
#    *** CORREZIONE ***
#    Applica il totale *solo* alle righe dove 'Ticker' è specificato
#    E 'QT. Attuale' NON è NaN (come da file di esempio).
for ticker, cols_to_sum in ticker_to_cols_map.items():
    
    # La maschera deve includere che QT. Attuale non sia nullo
    ticker_rows_mask = (df_final['Ticker'] == ticker) & (df_final['QT. Attuale'].notna())
    
    # Per quelle righe, imposta 'QT. totale' come la somma delle sue colonne di stato
    df_final.loc[ticker_rows_mask, 'QT. totale'] = df_final[cols_to_sum].sum(axis=1)

# 7. Pulisci le colonne di stato temporanee
df_final = df_final.drop(columns=state_cols)

# 8. Riordina le colonne per far corrispondere l'output di esempio
try:
    cols_order = ['Data', 'Conto', 'Ticker', 'QT. Attuale', 'QT. totale', 'Liquidita Attuale']
    df_final = df_final[cols_order]
except KeyError as e:
    print(f"Attenzione: la colonna {e} non è stata trovata...")
    pass

# 9. Salva il file finale
output_file = '/home/atonello/Downloads/portfolio_df_total_qt.csv'
df_final.to_csv(output_file, index=False)

File con 'QT. totale' calcolata (versione corretta) salvato come '/home/atonello/Downloads/portfolio_df_total_qt.csv'.

Verifica riga 7 (index 7) per 'AMD':
         Data    Conto Ticker  QT. Attuale  QT. totale  Liquidita Attuale
7  2025-06-10  BG Saxo    AMD          3.0         5.0               3.01

Verifica: La colonna 'QT. totale' calcolata ora corrisponde perfettamente a quella del file di esempio!
